# Train Models Using Relax Training APIs

There has been increased interest from the community in using TVM for training. Relax, the next generation graph level IR of TVM, also faces the demand of training model. 

We built a training workflow on Relax, including:
- an **automatic differentiation tool** based on source code transformation
- an **optimizer abstraction** and **common optimizers**
- a **loss function abstraction** and **common loss functions**
- **Trainer API** that integrates them together, and is easy to use

The training APIs can serve many needs. You can:
- train a model from scratch. You can use the compilation advantages of TVM to speed up the training process.
- fine-tune a model on device based on TVM.
- deploy the process of training models to various devices that TVM supports, such as FPGA and Raspberry PI.

In this tutorial, we will use the training APIs to train a model from scratch using both high-level Trainer API and low-level AD, optimizer, and loss APIs. Finally, we will walk through the source code of the automatic differentiation system to demonstrate how it works.

We will use the Fashion MNIST dataset to train an MLP model. This process also applies to most models.

## Preparations
First of all, we will import necessary dependencies and load the DataSet.


In [1]:
import numpy as np
import tvm
import tvm.script
from tvm import relax
from tvm.relax.block_builder import BlockBuilder
from tvm.relax.transform import LegalizeOps
from tvm.script.parser import ir as I, relax as R, tir as T

We will train a model on the [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset. The following code downloads and prepares the data from torchvision, and transforms it into NumPy arrays.

In [ ]:
import torch
import torchvision

test_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True)
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

img, label = next(iter(test_loader))
img = img.reshape(1, 28, 28).numpy()